# Lab 3: Source and Slink

## Tổng quan bài tập
**Đề bài**: Dựa vào tập dữ liệu, hãy đọc dữ liệu từ Dataset và tạo Schema hợp lý và phân vùng lại dữ liệu. Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán trên.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1X-6xQafkMwDU39xBfEdItDwqg8N_6WV9/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv sẽ có cấu trúc như sau:
```
root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
```

# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,046 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,305 kB]
Hit:13 https://ppa.launchpadcontent.net

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Source and Slink

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType
from pyspark.sql.functions import spark_partition_id

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('lab3')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/My Drive/DEP303/flight-time.csv'

Đọc dữ liệu bằng cách sử dụng StructType Schema

In [ ]:
flightSchemaStruct = StructType([
  StructField('FL_DATE', DateType()),
  StructField('OP_CARRIER', StringType()),
  StructField('OP_CARRIER_FL_NUM', IntegerType()),
  StructField('ORIGIN', StringType()),
  StructField('ORIGIN_CITY_NAME', StringType()),
  StructField('DEST', StringType()),
  StructField('DEST_CITY_NAME', StringType()),
  StructField('CRS_DEP_TIME', IntegerType()),
  StructField('DEP_TIME', IntegerType()),
  StructField('WHEELS_ON', IntegerType()),
  StructField('TAXI_IN', IntegerType()),
  StructField('CRS_ARR_TIME', IntegerType()),
  StructField('ARR_TIME', IntegerType()),
  StructField('CANCELLED', IntegerType()),
  StructField('DISTANCE', IntegerType())
]);

print('Schema by StructType')
flightTimeCsvDF = spark.read \
    .format("CSV") \
    .option("header", "true") \
    .schema(flightSchemaStruct) \
    .option("mode", "FAILFAST") \
    .option("dateFormat", "M/d/yyyy") \
    .load(DATASET_PATH);

flightTimeCsvDF.show(5)

Schema by StructType
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    16

Đọc dữ liệu bằng cách sử dụng String Schema

In [ ]:
flightSchemaDDL = """
  FL_DATE DATE,
  OP_CARRIER STRING,
  OP_CARRIER_FL_NUM INT,
  ORIGIN STRING,
  ORIGIN_CITY_NAME STRING,
  DEST STRING,
  DEST_CITY_NAME STRING,
  CRS_DEPT_TIME INT,
  DEP_TIME INT,
  WHEELS_ON INT,
  TAXI_IN INT,
  CRS_ARR_TIME INT,
  ARR_TIME INT,
  CANCELLED INT,
  DISTANCE INT
"""

print('Schema by String')
flightTimeCsvDF = spark.read \
    .format("CSV") \
    .option("header", "true") \
    .schema(flightSchemaDDL) \
    .option("mode", "FAILFAST") \
    .option("dateFormat", "M/d/yyyy") \
    .load(DATASET_PATH)


flightTimeCsvDF.show(5)

Schema by String
+----------+----------+-----------------+------+----------------+----+--------------+-------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEPT_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+-------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|         1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|         1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|         1415|    1414|     1642|      9|        1721|    

Tiếp theo đó bạn cần phải phân vùng lại cho dữ liệu và lưu dữ liệu đó ra file. Đầu tiên hãy phân vùng dữ liệu thành **5** vùng.

In [ ]:
flightTimeCsvDF.groupBy(spark_partition_id()).count().show()
print("Num Partitions before: " + str(flightTimeCsvDF.rdd.getNumPartitions()))


partitionedDF = flightTimeCsvDF.repartition(5)
print("Num Partitions after: " + str(partitionedDF.rdd.getNumPartitions()))
partitionedDF.groupBy(spark_partition_id()).count().show()

partitionedDF.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "./dataSink/avro/") \
    .save()

+--------------------+------+
|SPARK_PARTITION_ID()| count|
+--------------------+------+
|                   0|470477|
+--------------------+------+

Num Partitions before: 1
Num Partitions after: 5
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|94096|
|                   1|94095|
|                   2|94095|
|                   3|94095|
|                   4|94096|
+--------------------+-----+



Ngoài ra, bạn có có thể phân vùng lại theo các trường. Hãy hoàn thiện đoạn code để phân vùng theo hai trường là `"OP_CARRIER"` và `"ORIGIN"`

In [ ]:
flightTimeCsvDF.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataSink/json/") \
    .partitionBy(["OP_CARRIER", "ORIGIN"]) \
    .option("maxRecordsPerFile", 10000) \
    .save()